In [1]:
%autosave 0

Autosave disabled


In [2]:
import warnings
warnings.filterwarnings('ignore')
# For ignoring warnings printed by R magic commands

In [3]:
import os
import glob

In [4]:
# Import pandas
import pandas as pd

In [5]:
# Load R magic
%load_ext rpy2.ipython
# Import ggplot2
%R require(ggplot2);

In [6]:
# If experiment parameters change, modify this function
def AddParametersAsColumns(df, folder):
    expParams = folder.split('.')
    cloudParams = expParams[1].split('_')
    optimizerParams = expParams[2].split('_')
    df['groupSizeDist'] = cloudParams[9]
    df['placementDist'] = cloudParams[10]
    df['colocateNumHostsPerLeaf'] = int(cloudParams[11])
    df['seed'] = int(cloudParams[14])
    df['algorithm'] = optimizerParams[1]
    df['numBitmaps'] = int(optimizerParams[2])
    df['numLeafsPerBitmap'] = int(optimizerParams[3])
    df['redundancyPerBitmap'] = int(optimizerParams[4])
    df['numRulesPerLeaf'] = int(optimizerParams[5])
    
# Turns all files of a given filname across a set of folders into a single dataframe
def DataSetAsDataFrame(filename, folders, headers, header=None):
    # Read dataset as dataframe
    def ReadDataSet(folder):
        df = pd.read_csv(folder + '/' + filename, sep=',', header=header, names=headers)
        AddParametersAsColumns(df, folder)
        return df
    # Get the list of dataframes
    dfs = map(ReadDataSet, folders)
    # Combine into a single dataframe
    df = pd.concat(dfs)
    return df

In [7]:
%%R 
plotTheme <- theme(legend.title=element_blank(), legend.position="top", legend.text=element_text(size=6))

In [8]:
os.chdir('/mnt/sdb1/baseerat/numerical-evals/12-4-2017/logs-100K-random')
# folders = glob.glob("logs*_0.*")
folders = glob.glob("dynamic-logs.576_48_20_3000_10_5000_expon_100000_5_uniform_colocate-random-random_12_True_5_0_False.1_exact-match_30_3_0_64000_2_3.1000000")

In [9]:
switch_event_types_to_update_count = DataSetAsDataFrame('switch_event_types_to_update_count.csv',
                                                       folders, ['updates', 'switch', 'event'],
                                                       header=0)

In [10]:
switch_event_types_to_update_count.head()

,updates,switch,event,groupSizeDist,placementDist,colocateNumHostsPerLeaf,seed,algorithm,numBitmaps,numLeafsPerBitmap,redundancyPerBitmap,numRulesPerLeaf
0,1,virtual,join,uniform,colocate-random-random,12,0,exact-match,30,3,0,64000
1,46,virtual,join,uniform,colocate-random-random,12,0,exact-match,30,3,0,64000
2,63,virtual,join,uniform,colocate-random-random,12,0,exact-match,30,3,0,64000
3,88,virtual,join,uniform,colocate-random-random,12,0,exact-match,30,3,0,64000
4,31,virtual,join,uniform,colocate-random-random,12,0,exact-match,30,3,0,64000


In [22]:
switch_event_types_to_update_count[
    (switch_event_types_to_update_count['switch'] == 'virtual') & (switch_event_types_to_update_count['event'] == 'join')
]['updates'].describe()

count    475076.000000
mean        792.283919
std         770.057748
min           1.000000
25%           1.000000
50%         696.000000
75%        1451.000000
max        2568.000000
Name: updates, dtype: float64

In [21]:
switch_event_types_to_update_count[
    (switch_event_types_to_update_count['switch'] == 'virtual') & (switch_event_types_to_update_count['event'] == 'leave')
]['updates'].describe()

count    477283.000000
mean       1184.427572
std         650.280623
min           1.000000
25%         690.000000
50%        1216.000000
75%        1687.000000
max        2568.000000
Name: updates, dtype: float64

In [23]:
switch_event_types_to_update_count[
    (switch_event_types_to_update_count['switch'] == 'leaf') & (switch_event_types_to_update_count['event'] == 'join')
]['updates'].describe()

count    475076.000000
mean          3.634016
std          25.705681
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         274.000000
Name: updates, dtype: float64

In [24]:
switch_event_types_to_update_count[
    (switch_event_types_to_update_count['switch'] == 'leaf') & (switch_event_types_to_update_count['event'] == 'leave')
]['updates'].describe()

count    477283.000000
mean          5.504110
std          31.507638
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         274.000000
Name: updates, dtype: float64